In [2]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
from datetime import datetime, timedelta


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# data_1 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_account_x_balance.csv")
# data_2 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_party_products.csv")
# data_3 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_party_x_socdem.csv")
# data_4 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_story_logs.csv")
# data_5 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_story_texts.csv")
# data_6 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_test_transactions.csv")
data_7 = pd.read_csv("/content/drive/MyDrive/Хакатон/tinkoff_hackathon_data/avk_hackathon_data_train_transactions.csv")

In [5]:
# cast transaction_dttm to DateTime data type
data_7['transaction_dttm'] = pd.to_datetime(data_7['transaction_dttm'])

# HardCoded creation of dates [first date in every month of 2019]
unique_dates = np.array(['2019-01-01T00:00:00.000000000', '2019-02-01T00:00:00.000000000',
       '2019-03-01T00:00:00.000000000', '2019-04-01T00:00:00.000000000',
       '2019-05-01T00:00:00.000000000', '2019-06-01T00:00:00.000000000',
       '2019-07-01T00:00:00.000000000', '2019-08-01T00:00:00.000000000',
       '2019-09-01T00:00:00.000000000', '2019-10-01T00:00:00.000000000',
       '2019-11-01T00:00:00.000000000', '2019-12-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [6]:
# Constructing DataFrame of every possible combination tripplet (month,category,human)
# through Cartesian product [dates * categories * people_id] 
# There should be 12*37*50000 = 22 200 000 of such triplets

u_dat = pd.DataFrame(unique_dates, columns=['transaction_dttm'])
u_cat= pd.DataFrame(data_7.category.unique(), columns=['category'] )
u_id = pd.DataFrame(data_7.party_rk.unique(), columns=['party_rk'] )

u_dat['key'] = 1
u_cat['key'] = 1
u_id['key'] = 1

u_data7 = pd.merge(u_id, 
                  pd.merge(u_dat, u_cat, on ='key')).drop("key", 1)

# Added zero column for further easy concatation 
u_data7['transaction_amt_rur'] = 0
u_data7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22200000 entries, 0 to 22199999
Data columns (total 4 columns):
 #   Column               Dtype         
---  ------               -----         
 0   party_rk             int64         
 1   transaction_dttm     datetime64[ns]
 2   category             object        
 3   transaction_amt_rur  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 846.9+ MB


In [9]:
# Creating minimal sized DataFrame with columns = (transaction_dttm, human_id, category, price)
# Every month for every human should have all 37 categories

# This implemetation adds 0 expenditure for every month beginning to the main training DataFrame
df_main = data_7[['transaction_dttm', 'party_rk', 'category', 'transaction_amt_rur']]
df_main = pd.concat([df_main,u_data7])
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34187617 entries, 0 to 22199999
Data columns (total 4 columns):
 #   Column               Dtype         
---  ------               -----         
 0   transaction_dttm     datetime64[ns]
 1   party_rk             int64         
 2   category             object        
 3   transaction_amt_rur  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 1.3+ GB


In [14]:
df_main_sort = df_main.sort_values(by=['transaction_dttm','party_rk'])
df_main_sort

,transaction_dttm,party_rk,category,transaction_amt_rur
13748904,2019-01-01,1,Сувениры,0.0
13748905,2019-01-01,1,Фаст Фуд,0.0
13748906,2019-01-01,1,Супермаркеты,0.0
13748907,2019-01-01,1,Дом/Ремонт,0.0
13748908,2019-01-01,1,Сервисные услуги,0.0
...,...,...,...,...
11530073,2019-12-31,94575,Одежда/Обувь,3860.0
11607072,2019-12-31,94575,Супермаркеты,895.0
11921613,2019-12-31,94575,Фаст Фуд,592.0
11930989,2019-12-31,94575,NaN,10100.0


In [ ]:
df_main_sort.to_csv("df_main_sort.csv", index=False)